In [10]:
import sys
sys.path.append('D:\PyCharmProjects\VFPUMC02')
sys.path.append(r'C:\Users\Administrator\PycharmProjects\VFPUMC02')
sys.path.append(r'/root/VFPUMC02')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from datasets.DataSet import BankDataset
from utils.DataProcessUtils import split_labeled_unlabeled, vertical_split_array
from utils.DataProcessUtils import find_rounds_math
from utils.FateUtils import evaluate_model
from classfiers.VF_TwoStep import VF_TwoStep
from vf4lr.VF_LR import VF_LR
from classfiers.VF_LinearRegression import VF_LinearRegression
from enums.ModelType import ModelType

In [12]:
bankDataset = BankDataset()
X, y = bankDataset.get_data()
X, y_C, y_R = X[:,:-1], y, X[:,-1]
XA, XB = vertical_split_array(X,random_state=24,first_col_rate=0.5)

2025-01-21 21:08:32,980 - INFO - 正在加载数据集 bank.csv
2025-01-21 21:08:33,206 - INFO - 数据集 bank.csv 加载完成，数据集形状为 (41188, 64)


In [14]:
labeled, unlabeled = split_labeled_unlabeled(XA,XB,y_C,y_R,random_state=24,k=0.2)

总样本数: 41188
输入数组 0 的形状: (41188, 31)
输入数组 1 的形状: (41188, 31)
输入数组 2 的形状: (41188,)
输入数组 3 的形状: (41188,)
选择 8237 个样本作为有标签数据，剩余样本作为无标签数据
输入数组 0 的有标签数据形状: (8237, 31)
输入数组 0 的无标签数据形状: (32951, 31)
输入数组 1 的有标签数据形状: (8237, 31)
输入数组 1 的无标签数据形状: (32951, 31)
输入数组 2 的有标签数据形状: (8237,)
输入数组 2 的无标签数据形状: (32951,)
输入数组 3 的有标签数据形状: (8237,)
输入数组 3 的无标签数据形状: (32951,)


In [15]:
# 解包 labeled（有标签数据）
XA_L, XB_L, y_C_L, y_R_L = labeled
# 解包 unlabeled（无标签数据
XA_U, XB_U, y_C_U, y_R_U = unlabeled

In [19]:
vf_clf = VF_LR(
    learning_rate=0.2,
    epoch_num=10,
    batch_size=128
)
vf_reg = VF_LinearRegression(
    config={
    'batch_size': 32,
    'lr': 0.01,
    'lambda': 0.1,
    'n_iters':10
    }
)

2025-01-21 21:22:40,986 - ConsoleLogger - INFO - VF_LinearRegression 模型初始化完成。
2025-01-21 21:22:40,986 - INFO - VF_LinearRegression 模型初始化完成。
2025-01-21 21:22:40,992 - ConsoleLogger - INFO - 模型配置: {'batch_size': 32, 'lr': 0.01, 'lambda': 0.1, 'n_iters': 10}
2025-01-21 21:22:40,992 - INFO - 模型配置: {'batch_size': 32, 'lr': 0.01, 'lambda': 0.1, 'n_iters': 10}


In [21]:
vf_two_step = VF_TwoStep(
    clf=vf_clf,
    reg=vf_reg,
    k=0.1,
    max_iter=100
)

2025-01-21 21:24:59,682 - INFO - VF_TwoStep 类已成功初始化: max_iter=100, k=0.10, min_confidence=0.00, convergence_threshold=0, clf=VF_LR, reg=VF_LinearRegression


In [22]:
vf_two_step.fit?

Signature: vf_two_step.fit(XA_L, XB_L, y_L, XA_U, XB_U)
Docstring:
训练 TwoStep 模型。

参数:
- XA_L (numpy ndarray): 有标签数据的特征矩阵 A，形状为 (n_labeled_samples, n_features_A)。
- XB_L (numpy ndarray): 有标签数据的特征矩阵 B，形状为 (n_labeled_samples, n_features_B)。
- y_L (numpy ndarray): 有标签数据的标签向量，形状为 (n_labeled_samples, )。
- XA_U (numpy ndarray): 无标签数据的特征矩阵 A，形状为 (n_unlabeled_samples, n_features_A)。
- XB_U (numpy ndarray): 无标签数据的特征矩阵 B，形状为 (n_unlabeled_samples, n_features_B)。

返回:
- self: 训练后的模型实例。
File:      ~/VFPUMC02/classfiers/VF_TwoStep.py
Type:      method

In [23]:
vf_two_step.fit(
    XA_L=XA_L,
    XB_L=XB_L,
    y_L=y_C_L,
    XA_U=XA_U,
    XB_U=XB_U
)

2025-01-21 21:26:06,931 - INFO - 有标签数据 (L): 8237 样本
2025-01-21 21:26:06,932 - INFO - 无标签数据 (U): 32951 样本
2025-01-21 21:26:06,933 - INFO - 任务类型判定结果: classification
2025-01-21 21:26:06,933 - INFO - 这是一个分类任务。
2025-01-21 21:26:06,934 - INFO - 分类任务中的类别: [0 1]
2025-01-21 21:26:06,935 - INFO - [INIT] 初始化 self.pred, 未标注数据数量=32951
2025-01-21 21:26:06,935 - INFO - [INIT] 最大迭代次数 max_iter=100, 每轮选取比例 k=0.10
2025-01-21 21:26:06,935 - INFO - ===== [Epoch 1/100] 迭代开始 =====
2025-01-21 21:26:06,936 - INFO - [INFO] 当前 Labeled 样本数量: 8237, Unlabeled 样本数量: 32951
2025-01-21 21:26:06,936 - INFO - [STEP 1] 开始训练分类器 (基于 8237 个 Labeled 样本)...
2025-01-21 21:26:06,937 - ConsoleLogger - INFO - 开始训练垂直联邦逻辑回归模型，客户端A训练数据形状: (8237, 31), 客户端B训练数据形状: (8237, 31), 标签大小: 8237
2025-01-21 21:26:06,937 - INFO - 开始训练垂直联邦逻辑回归模型，客户端A训练数据形状: (8237, 31), 客户端B训练数据形状: (8237, 31), 标签大小: 8237
2025-01-21 21:26:06,938 - ConsoleLogger - INFO - 检测到的类别数量: 2
2025-01-21 21:26:06,938 - INFO - 检测到的类别数量: 2
2025-01-21 21:26:06,939 - ConsoleLogger 

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-21 21:26:07,149 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-21 21:26:07,149 - INFO - 第 2 轮训练完成。
2025-01-21 21:26:07,150 - ConsoleLogger - INFO - 开始第 3/10 轮训练...
2025-01-21 21:26:07,150 - INFO - 开始第 3/10 轮训练...
2025-01-21 21:26:07,251 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-21 21:26:07,251 - INFO - 第 3 轮训练完成。
2025-01-21 21:26:07,252 - ConsoleLogger - INFO - 开始第 4/10 轮训练...
2025-01-21 21:26:07,252 - INFO - 开始第 4/10 轮训练...
2025-01-21 21:26:07,350 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-21 21:26:07,350 - INFO - 第 4 轮训练完成。
2025-01-21 21:26:07,351 - ConsoleLogger - INFO - 开始第 5/10 轮训练...
2025-01-21 21:26:07,351 - INFO - 开始第 5/10 轮训练...
2025-01-21 21:26:07,449 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-21 21:26:07,449 - INFO - 第 5 轮训练完成。
2025-01-21 21:26:07,450 - ConsoleLogger - INFO - 开始第 6/10 轮训练...
2025-01-21 21:26:07,450 - INFO - 开始第 6/10 轮训练...
2025-01-21 21:26:07,548 - ConsoleLogger - INFO - 第 6 轮训练完成。
2025-01-21 21:26:07,548 - INFO - 第 6 轮训练完成。
2025-01-21 21:26:07,549 

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-21 21:26:08,665 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-21 21:26:08,665 - INFO - 第 2 轮训练完成。
2025-01-21 21:26:08,666 - ConsoleLogger - INFO - 开始第 3/10 轮训练...
2025-01-21 21:26:08,666 - INFO - 开始第 3/10 轮训练...
2025-01-21 21:26:08,801 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-21 21:26:08,801 - INFO - 第 3 轮训练完成。
2025-01-21 21:26:08,802 - ConsoleLogger - INFO - 开始第 4/10 轮训练...
2025-01-21 21:26:08,802 - INFO - 开始第 4/10 轮训练...
2025-01-21 21:26:08,937 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-21 21:26:08,937 - INFO - 第 4 轮训练完成。
2025-01-21 21:26:08,938 - ConsoleLogger - INFO - 开始第 5/10 轮训练...
2025-01-21 21:26:08,938 - INFO - 开始第 5/10 轮训练...
2025-01-21 21:26:09,072 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-21 21:26:09,072 - INFO - 第 5 轮训练完成。
2025-01-21 21:26:09,073 - ConsoleLogger - INFO - 开始第 6/10 轮训练...
2025-01-21 21:26:09,073 - INFO - 开始第 6/10 轮训练...
2025-01-21 21:26:09,206 - ConsoleLogger - INFO - 第 6 轮训练完成。
2025-01-21 21:26:09,206 - INFO - 第 6 轮训练完成。
2025-01-21 21:26:09,207 

...

2025-01-21 21:32:46,288 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-21 21:32:46,288 - INFO - 第 1 轮训练完成。
2025-01-21 21:32:46,289 - ConsoleLogger - INFO - 开始第 2/10 轮训练...
2025-01-21 21:32:46,289 - INFO - 开始第 2/10 轮训练...
2025-01-21 21:32:46,774 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-21 21:32:46,774 - INFO - 第 2 轮训练完成。
2025-01-21 21:32:46,775 - ConsoleLogger - INFO - 开始第 3/10 轮训练...
2025-01-21 21:32:46,775 - INFO - 开始第 3/10 轮训练...
2025-01-21 21:32:47,254 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-21 21:32:47,254 - INFO - 第 3 轮训练完成。
2025-01-21 21:32:47,255 - ConsoleLogger - INFO - 开始第 4/10 轮训练...
2025-01-21 21:32:47,255 - INFO - 开始第 4/10 轮训练...
2025-01-21 21:32:47,734 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-21 21:32:47,734 - INFO - 第 4 轮训练完成。
2025-01-21 21:32:47,735 - ConsoleLogger - INFO - 开始第 5/10 轮训练...
2025-01-21 21:32:47,735 - INFO - 开始第 5/10 轮训练...
2025-01-21 21:32:48,221 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-21 21:32:48,221 - INFO - 第 5 轮训练完成。
2025-01-21 21:32:48,222 

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-21 21:32:51,173 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-21 21:32:51,173 - INFO - 第 1 轮训练完成。
2025-01-21 21:32:51,174 - ConsoleLogger - INFO - 开始第 2/10 轮训练...
2025-01-21 21:32:51,174 - INFO - 开始第 2/10 轮训练...
2025-01-21 21:32:51,654 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-21 21:32:51,654 - INFO - 第 2 轮训练完成。
2025-01-21 21:32:51,655 - ConsoleLogger - INFO - 开始第 3/10 轮训练...
2025-01-21 21:32:51,655 - INFO - 开始第 3/10 轮训练...
2025-01-21 21:32:52,129 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-21 21:32:52,129 - INFO - 第 3 轮训练完成。
2025-01-21 21:32:52,130 - ConsoleLogger - INFO - 开始第 4/10 轮训练...
2025-01-21 21:32:52,130 - INFO - 开始第 4/10 轮训练...
2025-01-21 21:32:52,604 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-21 21:32:52,604 - INFO - 第 4 轮训练完成。
2025-01-21 21:32:52,605 - ConsoleLogger - INFO - 开始第 5/10 轮训练...
2025-01-21 21:32:52,605 - INFO - 开始第 5/10 轮训练...
2025-01-21 21:32:53,083 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-21 21:32:53,083 - INFO - 第 5 轮训练完成。
2025-01-21 21:32:53,084 

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-21 21:32:56,010 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-21 21:32:56,010 - INFO - 第 1 轮训练完成。
2025-01-21 21:32:56,011 - ConsoleLogger - INFO - 开始第 2/10 轮训练...
2025-01-21 21:32:56,011 - INFO - 开始第 2/10 轮训练...
2025-01-21 21:32:56,487 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-21 21:32:56,487 - INFO - 第 2 轮训练完成。
2025-01-21 21:32:56,488 - ConsoleLogger - INFO - 开始第 3/10 轮训练...
2025-01-21 21:32:56,488 - INFO - 开始第 3/10 轮训练...
2025-01-21 21:32:56,969 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-21 21:32:56,969 - INFO - 第 3 轮训练完成。
2025-01-21 21:32:56,970 - ConsoleLogger - INFO - 开始第 4/10 轮训练...
2025-01-21 21:32:56,970 - INFO - 开始第 4/10 轮训练...
2025-01-21 21:32:57,444 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-21 21:32:57,444 - INFO - 第 4 轮训练完成。
2025-01-21 21:32:57,445 - ConsoleLogger - INFO - 开始第 5/10 轮训练...
2025-01-21 21:32:57,445 - INFO - 开始第 5/10 轮训练...
2025-01-21 21:32:57,927 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-21 21:32:57,927 - INFO - 第 5 轮训练完成。
2025-01-21 21:32:57,928 

In [29]:
y_C_pred = vf_two_step.get_unlabeled_predict(ModelType.CLASSIFICATION)

In [31]:
evaluate_model(y_C_U,y_C_pred)


========== Classification Metrics ==========
Accuracy         : 0.1117
Precision (macro): 0.0559
Recall (macro)   : 0.5000
F1 (macro)       : 0.1005

Confusion Matrix:
[[    0 29269]
 [    0  3682]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     29269
           1       0.11      1.00      0.20      3682

    accuracy                           0.11     32951
   macro avg       0.06      0.50      0.10     32951
weighted avg       0.01      0.11      0.02     32951



In [ ]:
vf_two_step.fit(
    XA_L=XA_L,
    XB_L=XB_L,
    y_L=y_R_L,
    XA_U=XA_U,
    XB_U=XB_U
)

In [35]:
y_R_pred = vf_two_step.get_unlabeled_predict(ModelType.REGRESSION)

In [37]:
evaluate_model(y_R_U, y_R_pred, model_type=ModelType.REGRESSION)


========== Regression Metrics ==========
MSE               : 0.8393
RMSE              : 0.9161
MAE               : 0.7747
R^2 (Coefficient) : 0.1604
Explained Variance: 0.1605
